In [1]:
## import libaries
import pandas as pd
import numpy as np
import cv2
import os, sys
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
NB_EPOCHS = 30
BAT_SIZE = 32
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 120

In [3]:
## load data
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

In [4]:
le = LabelEncoder()
train['label_enc'] = le.fit_transform(train.label)
train.head(5)

,image_id,label,label_enc
0,train_1a,rice,17
1,train_1b,candy,2
2,train_1c,jam,11
3,train_1d,coffee,6
4,train_2a,vinegar,23


In [5]:
# function to read image
def read_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (IM_WIDTH, IM_HEIGHT))
    return img

In [6]:
## set path for images
TRAIN_PATH = 'input/train_img/'
TEST_PATH = 'input/test_img/'

In [7]:
# load data
train_img, test_img = [],[]
for img_path in tqdm(train['image_id'].values):
    train_img.append(read_img(TRAIN_PATH + img_path + '.png'))

for img_path in tqdm(test['image_id'].values):
    test_img.append(read_img(TEST_PATH + img_path + '.png'))

100%|██████████| 1732/1732 [00:04<00:00, 378.24it/s]


In [8]:
# normalize images
x_train = np.array(train_img, np.float32)
x_test = np.array(test_img, np.float32) 

In [9]:
x_train.shape

(3215, 299, 299, 3)

In [10]:
y_train = to_categorical(train.label_enc.values)

In [11]:
y_train.shape

(3215, 25)

In [12]:
#Transfer learning with Inception V3 
#include_top=False excludes final FC layer
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IM_WIDTH, IM_HEIGHT, 3))  


In [13]:
## set model architechture 
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(FC_SIZE, activation='relu')(x) #new FC layer, random init
predictions = Dense(y_train.shape[1], activation='softmax')(x) #new softmax layer
model = Model(input=base_model.input, output=predictions)

for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#model.summary()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [14]:
#batch_size = 32 # tune it
#epochs = 9 # increase it
train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True
)

In [15]:
train_generator = train_datagen.flow(
    x_train, y_train,
    #target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=BAT_SIZE,
)

In [16]:
history_tl = model.fit_generator(
    train_generator,
    nb_epoch=NB_EPOCHS,
    steps_per_epoch=x_train.shape[0]/BAT_SIZE,     
    class_weight='auto')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., class_weight="auto", steps_per_epoch=100.46875, epochs=30)`
  """


Epoch 1/30
101/100 [==============================] - 58s - loss: 3.2800 - acc: 0.1340    
Epoch 2/30
101/100 [==============================] - 56s - loss: 2.3746 - acc: 0.2808    
Epoch 3/30
101/100 [==============================] - 46s - loss: 2.0682 - acc: 0.3699    
Epoch 4/30
101/100 [==============================] - 48s - loss: 1.9002 - acc: 0.4227    
Epoch 5/30
101/100 [==============================] - 45s - loss: 1.7616 - acc: 0.4517    
Epoch 6/30
101/100 [==============================] - 48s - loss: 1.6460 - acc: 0.4951    
Epoch 7/30
101/100 [==============================] - 45s - loss: 1.5709 - acc: 0.5254    
Epoch 8/30
101/100 [==============================] - 45s - loss: 1.5105 - acc: 0.5284    
Epoch 9/30
101/100 [==============================] - 46s - loss: 1.4529 - acc: 0.5618    
Epoch 10/30
101/100 [==============================] - 45s - loss: 1.4181 - acc: 0.5587    
Epoch 11/30
101/100 [==============================] - 45s - loss: 1.3714 - acc: 0.5755  

In [17]:
for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
     layer.trainable = False
for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
     layer.trainable = True
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
history_ft = model.fit_generator(
    train_generator,
    nb_epoch=NB_EPOCHS,
    steps_per_epoch=x_train.shape[0]// BAT_SIZE,    
    class_weight='auto')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., class_weight="auto", steps_per_epoch=100, epochs=30)`
  """


Epoch 1/30
100/100 [==============================] - 87s - loss: 0.9140 - acc: 0.7114    
Epoch 2/30
100/100 [==============================] - 80s - loss: 0.6263 - acc: 0.8052    
Epoch 3/30
100/100 [==============================] - 80s - loss: 0.5453 - acc: 0.8277    
Epoch 4/30
100/100 [==============================] - 80s - loss: 0.4884 - acc: 0.8484    
Epoch 5/30
100/100 [==============================] - 80s - loss: 0.4325 - acc: 0.8653    
Epoch 6/30
100/100 [==============================] - 80s - loss: 0.4272 - acc: 0.8668    
Epoch 7/30
100/100 [==============================] - 80s - loss: 0.3763 - acc: 0.8921    
Epoch 8/30
100/100 [==============================] - 82s - loss: 0.3757 - acc: 0.8849    
Epoch 9/30
100/100 [==============================] - 87s - loss: 0.3415 - acc: 0.8886    
Epoch 10/30
100/100 [==============================] - 80s - loss: 0.3120 - acc: 0.9058    
Epoch 11/30
100/100 [==============================] - 80s - loss: 0.2920 - acc: 0.9091  

In [19]:
## predict test data
predictions = model.predict(preprocess_input(x_test))


In [20]:
np.argmax(predictions, axis=1)

array([ 2, 10, 23, ..., 21, 10, 13])

In [21]:
# get labels
predictions = np.argmax(predictions, axis=1)
pred_labels = le.inverse_transform(predictions)

In [22]:
pred_labels

array(['candy', 'honey', 'vinegar', ..., 'tea', 'honey', 'milk'], dtype=object)

In [23]:
## make submission
sub = pd.DataFrame({'image_id':test.image_id, 'label':pred_labels})
sub.to_csv('sub_Iv5.csv', index=False) ##70 

In [24]:
sub.label.value_counts()

candy          134
coffee         116
juice          110
chocolate      109
water          102
tea            100
cereal          99
jam             86
spices          74
chips           69
milk            65
honey           62
tomatosauce     58
nuts            57
rice            56
vinegar         55
soda            54
cake            52
beans           50
oil             46
pasta           45
fish            41
flour           32
sugar           31
corn            29
Name: label, dtype: int64